<a href="https://colab.research.google.com/github/ekohan/dotfiles/blob/master/hw05_part1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install scx

In [2]:
from scx.optimize import Model

In [3]:
plants = [
	{
		'name':'Mumbai',
		'capacity':4800,
	},
	{
		'name':'Chennai',
		'capacity':3500,
	},
	{
		'name':'Kochi',
		'capacity':2200,
	}
]

transportation = [
	{
		'origin':'Mumbai',
		'destination_dc':'Kalaburagi',
		'cost':16500
	},
	{
		'origin':'Mumbai',
		'destination_dc':'Bengaluru',
		'cost':10600
	},
	{
		'origin':'Chennai',
		'destination_dc':'Kalaburagi',
		'cost':12200
	},
	{
		'origin':'Chennai',
		'destination_dc':'Bengaluru',
		'cost':12600
	},
	{
		'origin':'Kochi',
		'destination_dc':'Kalaburagi',
		'cost':10300
	},
	{
		'origin':'Kochi',
		'destination_dc':'Bengaluru',
		'cost':9240
	}
]

demand = [
	{
		'dc':'Kalaburagi',
		'tractors':4800
	},
	{
		'dc':'Bengaluru',
		'tractors':5700
	}
]

In [10]:
for t in transportation:
  t['amt'] = Model.variable(name=f"{t['origin']}__{t['destination_dc']}__amt", lowBound=0, cat='Integer')

# Initialize the model
tadah_model = Model(name="HW05_TADAH", sense='minimize')

# Add objective
tadah_model.add_objective(
    fn=Model.sum([t['amt']*t['cost'] for t in transportation])
)

# Add constraints
# Demand
for d in demand:
  tadah_model.add_constraint(
    name=f"{d['dc']}__demand",
    fn=Model.sum([t['amt'] for t in transportation if t['destination_dc']==d['dc']]) >= d['tractors'],
  )
# Plant capacity
for p in plants:
  tadah_model.add_constraint(
      name=f"{p['name']}__capacity",
      fn=Model.sum([t['amt'] for t in transportation if t['origin']==p['name']]) <= p['capacity'],
  )


tadah_model.show_formulation()


HW05_TADAH:
MINIMIZE
12600*Chennai__Bengaluru__amt + 12200*Chennai__Kalaburagi__amt + 9240*Kochi__Bengaluru__amt + 10300*Kochi__Kalaburagi__amt + 10600*Mumbai__Bengaluru__amt + 16500*Mumbai__Kalaburagi__amt + 0
SUBJECT TO
Kalaburagi__demand: Chennai__Kalaburagi__amt + Kochi__Kalaburagi__amt
 + Mumbai__Kalaburagi__amt >= 4800

Bengaluru__demand: Chennai__Bengaluru__amt + Kochi__Bengaluru__amt
 + Mumbai__Bengaluru__amt >= 5700

Mumbai__capacity: Mumbai__Bengaluru__amt + Mumbai__Kalaburagi__amt <= 4800

Chennai__capacity: Chennai__Bengaluru__amt + Chennai__Kalaburagi__amt <= 3500

Kochi__capacity: Kochi__Bengaluru__amt + Kochi__Kalaburagi__amt <= 2200

VARIABLES
0 <= Chennai__Bengaluru__amt Integer
0 <= Chennai__Kalaburagi__amt Integer
0 <= Kochi__Bengaluru__amt Integer
0 <= Kochi__Kalaburagi__amt Integer
0 <= Mumbai__Bengaluru__amt Integer
0 <= Mumbai__Kalaburagi__amt Integer



In [12]:
tadah_model.show_outputs()

{'objective': 115286000.0,
 'status': 'Optimal',
 'variables': {'Chennai__Bengaluru__amt': 0.0,
               'Chennai__Kalaburagi__amt': 3500.0,
               'Kochi__Bengaluru__amt': 900.0,
               'Kochi__Kalaburagi__amt': 1300.0,
               'Mumbai__Bengaluru__amt': 4800.0,
               'Mumbai__Kalaburagi__amt': 0.0}}
